In [19]:
import os
import json
import base64
from tqdm import tqdm

In [20]:
def convert_to_int(points):
    numbers = points.split(",")
    new_points = [int(number) for number in numbers]
    return new_points

In [21]:
# 폴더를 가져옵니다.
raw_data_folder = r"C:\Users\Smart\Desktop\datas\emergency_vehicle\emergency_labeling_data\CCTV_data\labeling_data"
output_folder = r"C:\Users\Smart\Desktop\code\cctv_coverted"
img_folder = r"C:\Users\Smart\Desktop\datas\emergency_vehicle\emergency_img\img"

# 폴더의 파일 목록을 가져옵니다.
files = os.listdir(raw_data_folder)

# 폴더의 파일 목록을 순회합니다.
for file in tqdm(files):
    # 파일 이름을 가져옵니다.
    filename = file.split(".")[0]

     # 이미지 파일을 열어서 base64로 인코딩합니다.
    image_path = os.path.join(img_folder, filename + ".jpg")
    with open(image_path, "rb") as img_file:
        image_data = base64.b64encode(img_file.read()).decode()

    # 파일을 읽습니다.
    with open(os.path.join(raw_data_folder, file), "r",encoding="UTF-8") as f:
        data = json.load(f)

    labels = []
    shapes = []

    for i in range(len(data["row"])):
        label = data["row"][i]["label"]
        attributes1 = data["row"][i]["attributes1"]
        attributes2 = data["row"][i]["attributes2"]
        points1 = convert_to_int(data["row"][i]["points1"])
        # points2 = data["row"][i]["points2"]
        points3 = convert_to_int(data["row"][i]["points3"])
        # points4 = data["row"][i]["points4"]

        if label == "일반차량":
            label = "일반차량"
        elif label == "목적차량(특장차)":
            if attributes2 == "기타 특장차":
                label = "일반차량"
            elif attributes2 == "구급차":
                label = "ambulance"
            elif attributes2 == "경찰차":
                label = "police"
            elif attributes2 == "소방차":
                label = "fire_engine"

        labels.append({
            "label": label,
            "points":[ 
                    points1
                    # points2
                ,
                    points3
                    # points4
            ],
            "group_id": None,
            "description": "",
            "shape_type": "rectangle",
            "flags": {}
        })

    annotation = {
        "version": "5.2.1",
        "flags": {},
        "shapes": labels,
        "imagePath": filename + ".jpg",
        "imageData": image_data,
        "imageHeight": 1080,
        "imageWidth": 1920
    }

    # 라벨링 데이터를 다른 폴더에 .json형식으로 저장합니다.
    with open(os.path.join(output_folder, filename + ".json"), "w", encoding='UTF-8') as f:
        json.dump(annotation, f, ensure_ascii=False, indent=4)

100%|██████████| 3559/3559 [00:19<00:00, 182.26it/s]
